In [1]:
!pip install imutils
import cv2
import imutils
import numpy as np


You should consider upgrading via the '/home/lincoln/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
bg= None

In [3]:
def running_average(frame, weight):
    global bg

    if bg is None:
        bg= frame.copy().astype('float')
        return
    
    cv2.accumulateWeighted(frame, bg, weight)


In [4]:
def segmented_picture(frame, threshold=25):
    global bg

    abs_diff= cv2.absdiff(bg.astype("uint8"), frame)

    thresholded = cv2.threshold(abs_diff, threshold, 255, cv2.THRESH_BINARY)[1]

    cnts = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour= imutils.grab_contours(cnts)

    if len(contour)==0:
        return
    else:
        
        segment= max(contour, key= cv2.contourArea)
        return (thresholded, segment)

In [5]:
if __name__== "__main__":
    weight= 0.5

    video_cap= cv2.VideoCapture(0)

    num_frames= 0

    top, right, bottom, left= 10, 350, 225, 590

    while (video_cap.isOpened()):

        ret, frame= video_cap.read()
        if ret == True:

            frame= imutils.resize(frame, width= 700)

            frame= cv2.flip(frame, 1)

            clone= frame.copy()

            height, width= frame.shape[:2]

            region= frame[top:bottom, right:left]


            gray= cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)

            blur= cv2.GaussianBlur(gray, (5,5), 0)

            if num_frames < 30:
                running_average(blur, weight)

            else:
                hand = segmented_picture(blur)

                if hand is not None:

                    thresh, segment = hand


                    cv2.drawContours(clone, [segment + (right, top)], -1, (0,0,255))

                    cv2.imshow('Thresholded', thresh)

            cv2.rectangle(clone, (left, top), (right, bottom), (0, 255, 0), 2)

            num_frames +=1


            cv2.imshow("Live Feed", clone)

            if cv2.waitKey(1) & 0xFF == 27:
                break
            



video_cap.release()
cv2.destroyAllWindows()

            






